# Mahalanobis Outlier Algorithm Documentation

The aim of the document is to explain and document the algorithm used in Seldon's Mahalanobis Online Outlier Detector.

In the first part we give a high level overview of the algorithm, then we explain the implementation in detail.

## Overview

Outlier detection has many applications, ranging from preventing credit card fraud to detecting computer network intrusions. The available data is typically unlabeled and detection needs to be done in real-time. The outlier detector can be used as a standalone algorithm, or to detect anomalies in the input data of another predictive model.

The Mahalanobis outlier detection algorithm calculates an outlier score, which is a measure of distance from the center of the features distribution (Mahalanobis distance). If this outlier score is higher than a user-defined threshold, the observation is flagged as an outlier. The algorithm is online, which means that it starts without knowledge about the distribution of the features and learns as requests arrive. Consequently you should expect the output to be bad at the start and to improve over time.

As observations arrive, the algorithm will:
- Keep track and update the mean and sample covariance matrix of the dataset
- Apply a principal component analysis using these moments and project the new observations on the first 3 principal components (default value, can be changed)
- Compute the Mahalanobis distance from these projections to the projected mean
- Predict that the observation is an outlier if the Mahalanobis distance is larger than the threshold level

## Implementation

The core of the algorithm is using efficient and numerically stable streaming techniques to keep track of the mean and covariance matrix as new points are observed. The PCA is done by finding the eigenvectors of the covariance matrix using a function implemented in scipy. We also use an efficient algorithm to avoid inverting a new covariance matrix for each point in the batch.

### Object state

The outlier detector class has 9 attributes:

```python
class OutlierMahalanobis(object):
    def __init__(self,threshold=25,n_components=3,n_stdev=3,start_clip=50,max_n=-1):
        
        self.threshold = threshold
        self.n_components = n_components
        self.max_n = max_n
        self.n_stdev = n_stdev
        self.start_clip = start_clip
        
        self.clip = None
        self.mean = 0
        self.C = 0
        self.n = 0
```

- ***threshold***: If the Mahalanobis distance for an observation is above the threshold, the observation is classified as an outlier.
- ***n_components***: Number of principal components used for projection of the features.
- ***max_n***: Used to make the algorithm non stationary by capping the number of observations to max_n. When specified, the algorithm will behave like if it had seen at most max_n points, thus adapting faster to changes in the underlying distribution. Turned off (set to -1) by default.
- ***n_stdev***: Number of standard deviations away from the mean for each feature beyond which the feature's value is clipped before updating the mean and covariance matrix.
- ***start_clip***: Number of observations before feature-wise clipping is applied.
- ***clip***: List with lower and upper values for each feature beyond which clipping is applied after start_clip observations. Initiated with None.
- ***mean***: Online mean of the observed values.
- ***C***: Online covariance matrix of the observed values.
- ***n***: Number of observations so far.

### First Step: Tracking the mean and covariance matrix

```python
def predict(self,X,feature_names):
    """ Detect outliers using the Mahalanobis distance threshold. 

    Arguments:
        - X: input data
        - feature_names
    """

    nb = X.shape[0] # batch size
    p = X.shape[1] # number of features
    n_components = min(self.n_components,p)
    if self.max_n>0:
        n = min(self.n,self.max_n) # n can never be above max_n
    else:
        n = self.n

    # Clip X
    if self.n > self.start_clip:
        Xclip = np.clip(X,self.clip[0],self.clip[1])
    else:
        Xclip = X

    # Tracking the mean and covariance matrix
    roll_partial_means = Xclip.cumsum(axis=0)/(np.arange(nb)+1).reshape((nb,1))
    coefs = (np.arange(nb)+1.)/(np.arange(nb)+n+1.)
    new_means = self.mean + coefs.reshape((nb,1))*(roll_partial_means-self.mean)
    new_means_offset = np.empty_like(new_means)
    new_means_offset[0] = self.mean
    new_means_offset[1:] = new_means[:-1]

    coefs = ((n+np.arange(nb))/(n+np.arange(nb)+1.)).reshape((nb,1,1))
    B = coefs*np.matmul((Xclip - new_means_offset)[:,:,None],(Xclip - new_means_offset)[:,None,:])
    cov_batch = (n-1.)/(n+max(1,nb-1.))*self.C + 1./(n+max(1,nb-1.))*B.sum(axis=0)
```

Here we have implemented a numerically stable algorithm for updating the mean and covariance matrix based on the following formulas.

**Batch Online Mean**

Let $\bar{X}_n = \frac{1}{n} \sum_{k=1}^n{X_k} $ the rolling mean of $ (X_n)_n $

Let $\bar{X}_{n,N} = \frac{1}{N-n} \sum_{k=n+1}^N{X_k} $ the batch mean of $X_n$ between $n$ and $N$

Then we have: 

$ \bar{X}_{n+b} = \bar{X}_n + \frac{b}{n+b}(\bar{X}_{n,n+b} - \bar{X}_n) $ $ (1) $

**Batch Online Covariance Matrix**

Let $C_n = \frac{1}{n-1} \sum_{k=1}^n{(X_k - \bar{X}_n)(X_k - \bar{X}_n)^t} $ the rolling sample covariance matrix of $ (X_n)_n $

Then we have:

$ C_{n+b} = \frac{n-1}{n+b-1}C_{n} + \frac{1}{n+b-1}\sum^{b-1}_{i=0}\frac{n+i}{n+i+1}(X_{n+i+1}-\bar{X}_{n+i})(X_{n+i+1}-\bar{X}_{n+i})^t $ $ (2) $


The online mean and covariance matrix are updated with clipped values of new observations. As a result, we can limit the impact of outliers on the estimated mean and covariance matrix. This can be particularly helpful when outliers arrive in sequences instead of uniformly distributed over time. Clipping is applied to each feature that has a value beyond the lower or upper boundary defined by the *n_stdev* hyperparameter. 

The 2 figures below illustrate the impact of clipping on the detection of computer network intrusions by showing the outlier score per observation for a sequence of network data. Scores above the red threshold line are classified as outliers by the algorithm. Please check out the [case study](./outlier_mahalanobis.ipynb) for more information regarding the dataset. During the first 500 observations, the fraction of anomalies is set at 2%. We then increase the amount of network intrusions temporarily to 20% over the next 500 observations before settling at an anomaly rate of 5%. No clipping is applied in the first figure, while figure 2 clips observations 3 standard deviations away from the online mean of each feature. The higher fraction of outliers is quickly incorporated in the unclipped covariance matrix. Consequently, the Mahalanobis distances of both outliers and inliers become similar and it is hard to spot anomalies. The covariance matrix in the clipped outlier detector is less impacted by the anomalies. As a result, the Mahalanobis distance of outliers is much larger than for normal data and less affected by the temporary spike in outliers.

![outliers_unclipped](images/outliers_no_clipping.png)

![outliers_clipped](images/outliers_3stdev_clipped.png)

## Second Step: PCA and projection

```python
    # PCA
    eigvals, eigvects = eigh(cov_batch,eigvals=(p-n_components,p-1))

    # Projections
    proj_features = np.matmul(features,eigvects)
    proj_means = np.matmul(new_means_offset,eigvects)
    if type(self.C) == int and self.C == 0:
        proj_cov = np.diag(np.zeros(n_components))
    else:
        proj_cov = np.matmul(eigvects.transpose(),np.matmul(self.C,eigvects))
```

```python
    # PCA
    eigvals, eigvects = eigh(cov_batch,eigvals=(p-n_components,p-1))

    # Projections
    proj_x = np.matmul(X,eigvects)
    proj_x_clip = np.matmul(Xclip,eigvects)
    proj_means = np.matmul(new_means_offset,eigvects)
    if type(self.C) == int and self.C == 0:
        proj_cov = np.diag(np.zeros(n_components))
    else:
        proj_cov = np.matmul(eigvects.transpose(),np.matmul(self.C,eigvects))
```

We compute the first principal components: these are the eigenvectors of the sample covariance matrix associated to the largest eigenvalues. For this we use the function [eigh](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh.html) from ```scipy.linalg```.

We then project the new, both original and clipped, observations, the rolling means and the previous covariance matrix on the principal components subspace.

## Third Step: Outlier detection in the Principal Components Subspace

### Substep: Fast computation of the inverses of the covariance matrices

To compute the outlier score of each point in the new batch, we need the inverse of the covariance matrix of all the points up to this one. This means inverting $b$ matrices. We made this operation faster by leveraging the fact that each covariance matrix is a rank one update of the previous one. 
Knowing this we can use the following theorem.

**Theorem:**

if $A$ and $A+B$ are invertible and $rank(B) = 1$ then

$ (A + B)^{-1} = A^{-1} - \frac{1}{1+trace(BA^{-1})}A^{-1}BA^{-1} $

The implementation is:

```python
    coefs = (1./(n+np.arange(nb)+1.)).reshape((nb,1,1))
    B = coefs*np.matmul((proj_x_clip - proj_means)[:,:,None],(proj_x_clip - proj_means)[:,None,:])

    all_C_inv = np.zeros_like(B)
    c_inv = None
    _EPSILON = 1e-8

    for i, b in enumerate(B):
        if c_inv is None:
            if abs(np.linalg.det(proj_cov)) > _EPSILON:
                c_inv = np.linalg.inv(proj_cov)
                all_C_inv[i] = c_inv
                continue
            else:
                if n + i == 0:
                    continue
                proj_cov = (n + i -1. )/(n + i)*proj_cov + b
                continue
        else:
            c_inv = (n + i - 1.)/float(n + i - 2.)*all_C_inv[i-1]
        BC1 = np.matmul(B[i-1],c_inv)
        all_C_inv[i] = c_inv - 1./(1.+np.trace(BC1))*np.matmul(c_inv,BC1)
```

### Finally, we update the attributes including the clip ranges, compute the outlier scores and return the outlier predictions

```python
    self.mean = new_means[-1]
    self.C = cov_batch
    stdev = np.sqrt(np.diag(cov_batch))
    self.n += nb
    if self.n > self.start_clip:
        self.clip = [self.mean-self.n_stdev*stdev,self.mean+self.n_stdev*stdev]

    # Outlier scores and predictions
    x_diff = proj_x-proj_means
    self.score = np.matmul(x_diff[:,None,:],np.matmul(all_C_inv,x_diff[:,:,None])).reshape(nb)
    self.prediction = np.array([1 if s > self.threshold else 0 for s in self.score]).astype(int)
    
    return self.prediction
```

The outlier score is the Mahalanobis distance in the Principal Components Subspace.

$ score_n = (X_n - \bar{X}_{n-1})^tC_{n-1}^{-1}(X_n - \bar{X}_{n-1}) $